# Workgroup 4 

- Andrea Ulloa
- Angela Coapaza
- Ana Angulo

## 1. Causal Forest (R and Python script)

In this lab, we are going to replicate the results obtained in the paper of Athey and Wager (2018) in phyton. First, we download all the packages that we are going to use.

In [7]:
pip install syncrng


Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install plotnine

Note: you may need to restart the kernel to use updated packages.


In [15]:
import random
import pandas as pd
import numpy as np
from scipy.stats import norm

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy

from SyncRNG import SyncRNG
import numpy as np
import re
from statsmodels.sandbox.stats.multicomp import multipletests
from scipy import linalg
from itertools import chain

from SyncRNG import SyncRNG

from CTL.causal_tree_learn import CausalTree
from sklearn.model_selection import train_test_split
import plotnine as p
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [79]:
random.seed(1)

In [80]:
## Next, we load the data
dataall = pd.read_csv("../../../../data_lab/synthetic_data.csv")
dataall.shape[1]

13

In [88]:
dataall.head(5)

,schoolid,Z,Y,S3,C1,C2,C3,XC,X1,X2,X3,X4,X5
0,76,1,0.081602,6,4,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
1,76,1,-0.385869,4,12,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
2,76,1,0.398184,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
3,76,1,-0.175037,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
4,76,1,0.884583,6,4,1,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757


In [81]:
dataall.schoolid = np.multiply(dataall.schoolid,1)

In [82]:
school_id = pd.to_numeric(dataall.schoolid, errors='coerce').astype(int)

In [83]:
school_id

0        76
1        76
2        76
3        76
4        76
         ..
10386     1
10387     1
10388     1
10389     1
10390     1
Name: schoolid, Length: 10391, dtype: int32

In [84]:
school_size = school_id.sum(axis = 0, skipna = True)

In [85]:
school_size

414485

In [86]:
# It appears that school ID does not affect pscore. So ignore it
# in modeling, and just treat it as source of per-cluster error.


In [125]:
W = dataall.Z.reset_index( drop = True )
Y = dataall.Y.reset_index( drop = True )
X_raw = dataall.iloc[:, 3:].reset_index( drop = True )

In [126]:
X_raw

,S3,C1,C2,C3,XC,X1,X2,X3,X4,X5
0,6,4,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
1,4,12,2,1,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
2,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
3,6,4,2,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
4,6,4,1,0,4,0.334544,0.648586,-1.310927,0.224077,-0.426757
...,...,...,...,...,...,...,...,...,...,...
10386,7,4,2,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10387,7,4,2,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10388,2,15,1,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702
10389,5,4,1,1,3,1.185986,-1.129889,1.009875,1.005063,-1.174702


In [129]:
C1_exp = matrix_factorization ((X_raw.C1) + 0)

NameError: name 'matrix_factorization' is not defined